<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception
from keras.preprocessing.image import ImageDataGenerator
import os ,sys


In [31]:
#git clone https://github.com/gkothari25/Check_defect.git

In [32]:
ls

Check_defect.ipynb  README.md  test_dataset/  train_Defect/   training_dataset/
model/              test/      train/         train_Healthy/  wewe/


In [33]:
x_train = []
# Load the training data set by looping over every image file
for image_file in Path("training_dataset").glob("**/*.jpg"):
    #print(image_file)
    
    
    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)


print(len(x_train))

215


In [34]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

In [35]:
# Add a label for this image. If it was a Defect, label it 0. If it was a Healthy, label it 1.
y_train = []
path = "training_dataset/"
classes = ["Defect","Healthy"]
for i in classes:
    if i == "Defect":
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_train.append(0)
    else:
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_train.append(1)
print(len(y_train))

215


In [36]:
# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))
x = feature_extractor
#print(x)


In [37]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "model/x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "model/y_train.dat")


['model/y_train.dat']

In [38]:
# Load data set of extracted features
x_train = joblib.load("model/x_train.dat")
y_train = joblib.load("model/y_train.dat")


In [39]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [40]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [41]:
print(len(x_train))
print(len(y_train))

215
215


In [42]:
# Train the model
model.fit(
    x_train,
    y_train,
    validation_split=0.05,
    epochs=13,
    shuffle=True,
    verbose=2
)

# Save the trained model to a file so we can use it to make predictions later
model.save("model/Defect_healthy_classifier_model.h5")

Train on 204 samples, validate on 11 samples
Epoch 1/13
 - 2s - loss: 4.3979 - acc: 0.4853 - val_loss: 0.2732 - val_acc: 0.9091
Epoch 2/13
 - 0s - loss: 2.3515 - acc: 0.5637 - val_loss: 1.0388 - val_acc: 0.4545
Epoch 3/13
 - 0s - loss: 1.1563 - acc: 0.6176 - val_loss: 0.5293 - val_acc: 0.8182
Epoch 4/13
 - 0s - loss: 0.5706 - acc: 0.7059 - val_loss: 0.3263 - val_acc: 0.9091
Epoch 5/13
 - 0s - loss: 0.5676 - acc: 0.7206 - val_loss: 0.2560 - val_acc: 0.9091
Epoch 6/13
 - 0s - loss: 0.4977 - acc: 0.7353 - val_loss: 0.3896 - val_acc: 0.8182
Epoch 7/13
 - 0s - loss: 0.4631 - acc: 0.7941 - val_loss: 0.2760 - val_acc: 1.0000
Epoch 8/13
 - 0s - loss: 0.3607 - acc: 0.8284 - val_loss: 0.3119 - val_acc: 1.0000
Epoch 9/13
 - 0s - loss: 0.3571 - acc: 0.8284 - val_loss: 0.3699 - val_acc: 0.8182
Epoch 10/13
 - 0s - loss: 0.3155 - acc: 0.8578 - val_loss: 0.2319 - val_acc: 1.0000
Epoch 11/13
 - 0s - loss: 0.3484 - acc: 0.8578 - val_loss: 0.4954 - val_acc: 0.8182
Epoch 12/13
 - 0s - loss: 0.2873 - acc: 

In [43]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [44]:
#Add a label for this image. If it was a Defect, label it 0. If it was a Healthy, label it 1.
y_test = []
path = "test_dataset/"
classes = ["Defect","Healthy"]
for i in classes:
    if i == "Defect":
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_test.append(0)
    else:
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_test.append(1)


In [45]:
# Empty lists to hold the images and labels for each each image
x_test = []

# Load the test data set by looping over every image file
for image_file in Path("test_dataset").glob("**/*.jpg"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)
    
# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

print(x_test.shape)

(28, 73, 73, 3)


In [46]:
# Load our trained classifier model
model = load_model("model/Defect_healthy_classifier_model.h5")
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

print(features_x.shape)


(28, 3, 3, 2048)


In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)
#print(predictions)

# If the model is more than 50% sure the object is a Defect, call it a Defect.
# Otherwise, call it "Healthy".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions,labels=[0,1]).ravel()
print("True negative",tn)
print("False positive",fp)
print("False negative",fn)
print("True Positive",tp)

accuracy_score = accuracy_score(y_test, predictions)
print("Accuracy score percent is :",accuracy_score)

f1_score = f1_score(y_test, predictions)
print("f1_score percent is :",f1_score)

precision_score = precision_score(y_test, predictions)
print("precision_score percent is :",precision_score)

recall_score = recall_score(y_test, predictions)
print("recall_score percent is :",recall_score)

 

True negative 5
False positive 9
False negative 1
True Positive 13
Accuracy score percent is : 0.6428571428571429
f1_score percent is : 0.7222222222222223
precision_score percent is : 0.5909090909090909
recall_score percent is : 0.9285714285714286
